In [0]:
! unzip projectRE.zip

Archive:  projectRE.zip
  inflating: feature_engineering.py  
  inflating: LICENSE                 
  inflating: requirements.txt        
  inflating: fnc_kfold.py            
   creating: __MACOSX/
  inflating: __MACOSX/._fnc_kfold.py  
  inflating: embedding_matrixRE.npy  
   creating: features/
  inflating: features/hand.competition.npy  
  inflating: features/hand.9.npy     
  inflating: features/hand.8.npy     
 extracting: features/dummy.txt      
  inflating: features/overlap.competition.npy  
  inflating: features/overlap.9.npy  
  inflating: features/refuting.9.npy  
  inflating: features/refuting.8.npy  
  inflating: features/overlap.8.npy  
  inflating: features/polarity.7.npy  
  inflating: features/polarity.6.npy  
  inflating: features/polarity.competition.npy  
  inflating: features/polarity.4.npy  
  inflating: features/polarity.5.npy  
  inflating: features/polarity.1.npy  
  inflating: features/polarity.0.npy  
  inflating: features/polarity.2.npy  
  inflating: featu

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
import os
import numpy as np
from numpy import array
from numpy import asarray
from tensorflow.keras.utils import to_categorical
import pandas as pd
import tensorflow as tf
#from gensim.models import Word2Vec
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional, Flatten
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import callbacks, regularizers
from utils.dataset import DataSet
from utils.generate_test_splits import train_vali_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission
from tensorflow.keras.utils import plot_model
from utils.system import parse_params, check_version
import re
import string

In [0]:
train_stance = pd.read_csv('train_stances.csv')
train_body = pd.read_csv('train_bodies.csv')
compe_stance = pd.read_csv('competition_test_stances.csv')
compe_body = pd.read_csv('competition_test_bodies.csv')

In [0]:
train = train_stance.merge(train_body, how = 'left', on = 'Body ID')
compe = compe_stance.merge(compe_body,how = 'left', on = 'Body ID')

In [0]:
train.head()

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's..."


In [0]:
def stance_to_onehot(stance):
    li = []
    LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
    for i in stance:
        if i == LABELS[0]:
            li.append([1,0,0,0])
        elif i == LABELS[1]:
            li.append([0,1,0,0])
        elif i == LABELS[2]:
            li.append([0,0,1,0])
        elif i == LABELS[3]:
            li.append([0,0,0,1])
    return np.array(li)

In [0]:
token_pattern = r"(?u)\b\w\w+\b"
def get_tokens(line,
                    token_pattern=token_pattern):
    token_pattern = re.compile(token_pattern, flags = re.UNICODE)
    tokens = [x.lower() for x in token_pattern.findall(line)]
    
    return tokens

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_vali, y_train, y_vali = train_test_split(train, train['Stance'], random_state=10,shuffle =False,test_size =0.1)

In [0]:
train_headline = [get_tokens(text) for text in X_train['Headline'].tolist()]
train_body = [get_tokens(text) for text in X_train['articleBody'].tolist()]
vali_headline = [get_tokens(text) for text in X_vali['Headline'].tolist()]
vali_body = [get_tokens(text) for text in X_vali['articleBody'].tolist()]
compe_headline = [get_tokens(text) for text in compe['Headline'].tolist()]
compe_body = [get_tokens(text) for text in compe['articleBody'].tolist()]



In [136]:
tokenizer = Tokenizer()
corpus = train_headline+train_body+compe_headline+compe_body+vali_headline+vali_body
tokenizer.fit_on_texts([' '.join(seq) for seq in corpus])
vocab_size = len(tokenizer.word_index) + 1
vocab_size

29419

In [0]:
train_headline_seq = tokenizer.texts_to_sequences(train_headline)
train_body_seq = tokenizer.texts_to_sequences(train_body)
vali_headline_seq = tokenizer.texts_to_sequences(vali_headline)
vali_body_seq = tokenizer.texts_to_sequences(vali_body)

competition_headline_seq = tokenizer.texts_to_sequences(compe_headline)
competition_body_seq = tokenizer.texts_to_sequences(compe_body)

In [138]:
MAX_BODY_LENGTH = int(np.percentile([len(doc) for doc in train_body_seq+competition_body_seq],10))

print('10th Percentile body Length:', MAX_BODY_LENGTH)

10th Percentile body Length: 99


In [139]:
MAX_HEADLINE_LENGTH = int(np.percentile([len(doc) for doc in train_headline_seq+competition_headline_seq], 90))

print('90th Percentile headline Length:', MAX_HEADLINE_LENGTH)
MAX_HEADLINE_LENGTH =15
MAX_BODY_LENGTH = int(np.mean([len(doc) for doc in train_body_seq+competition_body_seq]))

print('90th Percentile body Length:', MAX_BODY_LENGTH)
MAX_BODY_TRUNC = 50

90th Percentile headline Length: 16
90th Percentile body Length: 353


In [0]:
train_headline = pad_sequences(train_headline_seq, maxlen=MAX_HEADLINE_LENGTH, padding='post', truncating='post')
train_first_body = pad_sequences(train_body_seq, maxlen=MAX_BODY_TRUNC, padding='post', truncating='post')
train_last_body = pad_sequences(train_body_seq, maxlen=MAX_BODY_TRUNC, padding='post', truncating='pre')

vali_headline = pad_sequences(vali_headline_seq, maxlen=MAX_HEADLINE_LENGTH, padding='post', truncating='post')
vali_first_body = pad_sequences(vali_body_seq, maxlen=MAX_BODY_TRUNC, padding='post', truncating='post')
vali_last_body = pad_sequences(vali_body_seq, maxlen=MAX_BODY_TRUNC, padding='post', truncating='pre')

competition_headline = pad_sequences(competition_headline_seq, maxlen=MAX_HEADLINE_LENGTH, padding='post', truncating='post')
competition_first_body = pad_sequences(competition_body_seq, maxlen=MAX_BODY_TRUNC, padding='post', truncating='post')
competition_last_body = pad_sequences(competition_body_seq, maxlen=MAX_BODY_TRUNC, padding='post', truncating='pre')



In [142]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.42B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1917494 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
del embeddings_index

In [0]:
def builtWeightSharingModel(drop_out_rate=0.5,dense_neuron = 32):
    headline_input = Input(shape=(MAX_HEADLINE_LENGTH,), name='headline_input')
    first_body_input = Input(shape=(MAX_BODY_TRUNC,), name='first_body_input')
    last_body_input = Input(shape=(MAX_BODY_TRUNC,), name='last_body_input')
    e = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='headline_word_embedding_layer', 
                          mask_zero=True)
    headline_embedding = e(headline_input)
    first_body_embedding = e(first_body_input)
    last_body_embedding = e(last_body_input)
    encoder_outputs, state_h, state_c = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_encoder')(headline_embedding)
    encoder_states = [state_h, state_c]
    decoder = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_first_body')
    first_body_outputs, _, _ = decoder(first_body_embedding, initial_state=encoder_states)
    last_body_outputs, _, _ = decoder(last_body_embedding, initial_state=encoder_states)
    x = tf.keras.layers.concatenate([first_body_outputs, last_body_outputs])
    x = Dropout(rate=drop_out_rate, name='dropout_0')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_2')(x)

    # And finally we add the main logistic regression layer
    main_output = Dense(4, activation='softmax', name='main_output')(x)
    model = Model(inputs=[headline_input, first_body_input, last_body_input], outputs=main_output)
    model.summary()
    return model

In [0]:
train_stance_onehot = stance_to_onehot(y_train)
vali_stance_onehot = stance_to_onehot(y_vali)
competition_stance_onehot = stance_to_onehot(compe['Stance'])


In [0]:
len(competition_stance_onehot)

49972

In [0]:
def trainModel(model,train_input,vali_input, batch=1000,ep=200):
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    output_directory = ''
    model_checkpoint = callbacks.ModelCheckpoint(os.path.join(output_directory , 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'),save_best_only=True,mode='auto',period=10)
    '''
    model.fit([np.concatenate((train_headline, hold_out_headline), axis=0), np.concatenate((train_first_body, hold_out_first_body), axis=0), np.concatenate((train_last_body, hold_out_last_body), axis=0)], np.concatenate((train_stance_onehot, hold_out_stance_onehot), axis=0),
          batch_size=batch,
          epochs=ep,
          validation_data=([competition_headline, competition_first_body, competition_last_body], competition_stance_onehot),callbacks=[model_checkpoint])
    '''
    model.fit(train_input, train_stance_onehot,
          batch_size=batch,
          epochs=ep,
          validation_data=(vali_input, vali_stance_onehot),callbacks=[model_checkpoint])
    #'''
    return model

In [0]:
LSTM_DIM = 300
EMBEDDING_DIM = 300

In [148]:
model = trainModel(builtWeightSharingModel(drop_out_rate=0.5),[train_headline, train_first_body, train_last_body],[vali_headline, vali_first_body, vali_last_body])

W0723 16:23:44.770649 140219664385920 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_body_input (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
headline_input (InputLayer)     [(None, 15)]         0                                            
__________________________________________________________________________________________________
headline_word_embedding_layer ( multiple             8825700     headline_input[0][0]             
                                                                 first_body_input[0][0]           
                                                                 last_body_input[0][0]            
____________________________________________________________________________________________

In [0]:
def y_to_stance(onehot):
    LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
    return [LABELS[index] for index in onehot]

def evaluate(predicted, actual):
    
    predicted = y_to_stance(predicted)
    print("Scores on the test set")
    report_score(actual,predicted)

In [152]:
pred = model.predict([vali_headline, vali_first_body,vali_last_body])
y_classes = pred.argmax(axis=-1)
actual = y_vali
evaluate(y_classes, actual)



pred = model.predict([competition_headline, competition_first_body, competition_last_body])
y_classes = pred.argmax(axis=-1)
actual = compe['Stance']
evaluate(y_classes, actual)



Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    346    |     6     |     5     |     5     |
-------------------------------------------------------------
| disagree  |     6     |    72     |     0     |     1     |
-------------------------------------------------------------
|  discuss  |     6     |     1     |    889    |     2     |
-------------------------------------------------------------
| unrelated |     8     |     1     |    21     |   3629    |
-------------------------------------------------------------
Score: 2220.25 out of 2253.75	(98.51358846367165%)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    871    |    38    

In [0]:
def builtCompositeModel(drop_out_rate=0.5,dense_neuron = 32):
    headline_input = Input(shape=(MAX_HEADLINE_LENGTH,), name='headline_input')
    first_body_input = Input(shape=(MAX_BODY_TRUNC,), name='first_body_input')
    last_body_input = Input(shape=(MAX_BODY_TRUNC,), name='last_body_input')
    headline_embedding = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='headline_word_embedding_layer', 
                          mask_zero=True)(headline_input)
    first_body_embedding = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='first_body_word_embedding_layer', 
                          mask_zero=True)(first_body_input)
    last_body_embedding = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='last_bodyword_embedding_layer', 
                          mask_zero=True)(last_body_input)
    encoder_outputs, state_h, state_c = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_encoder')(headline_embedding)
    encoder_states = [state_h, state_c]
    first_body_outputs, _, _ = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_first_body')(first_body_embedding, initial_state=encoder_states)
    last_body_outputs, _, _ = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_last_body')(last_body_embedding, initial_state=encoder_states)
    x = tf.keras.layers.concatenate([first_body_outputs, last_body_outputs])
    x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_2')(x)

    # And finally we add the main logistic regression layer
    main_output = Dense(4, activation='softmax', name='main_output')(x)
    model = Model(inputs=[headline_input, first_body_input, last_body_input], outputs=main_output)
    model.summary()
    return model

In [0]:
def builtConditionalModel(drop_out_rate=0.5,dense_neuron = 32):
    headline_input = Input(shape=(MAX_HEADLINE_LENGTH,), name='headline_input')
    first_body_input = Input(shape=(MAX_BODY_TRUNC,), name='first_body_input')
    #last_body_input = Input(shape=(MAX_BODY_TRUNC,), name='last_body_input')
    e = Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embedding_matrix], trainable=False, name='headline_word_embedding_layer', 
                          mask_zero=True)
    headline_embedding = e(headline_input)
    first_body_embedding = e(first_body_input)
    #last_body_embedding = e(last_body_input)
    encoder_outputs, state_h, state_c = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_encoder')(headline_embedding)
    encoder_states = [state_h, state_c]
    decoder = LSTM(LSTM_DIM, return_sequences=False, return_state = True,dropout=0.5,recurrent_dropout = 0.5, name='lstm_layer_first_body')
    first_body_outputs, _, _ = decoder(first_body_embedding, initial_state=encoder_states)
    #last_body_outputs, _, _ = decoder(last_body_embedding, initial_state=encoder_states)
    #x = tf.keras.layers.concatenate([first_body_outputs, last_body_outputs])
    x = Dropout(rate=drop_out_rate, name='dropout_0')(first_body_outputs)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_1')(x)
    #x = Dense(dense_neuron, activation='relu')(x)
    #x = Dropout(rate=drop_out_rate, name='dropout_2')(x)

    # And finally we add the main logistic regression layer
    main_output = Dense(4, activation='softmax', name='main_output')(x)
    model = Model(inputs=[headline_input, first_body_input], outputs=main_output)
    model.summary()
    return model

In [155]:
model = trainModel(builtConditionalModel(drop_out_rate=0.5),[train_headline, train_first_body],[vali_headline, vali_first_body])

W0723 17:31:31.077806 140219664385920 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_body_input (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
headline_input (InputLayer)     [(None, 15)]         0                                            
__________________________________________________________________________________________________
headline_word_embedding_layer ( multiple             8825700     headline_input[0][0]             
                                                                 first_body_input[0][0]           
__________________________________________________________________________________________________
lstm_layer_encoder (LSTM)       [(None, 300), (None, 721200      headline_word_embedding_lay

In [156]:
pred = model.predict([vali_headline, vali_first_body,vali_last_body])
y_classes = pred.argmax(axis=-1)
actual = y_vali
evaluate(y_classes, actual)



pred = model.predict([competition_headline, competition_first_body, competition_last_body])
y_classes = pred.argmax(axis=-1)
actual = compe['Stance']
evaluate(y_classes, actual)

Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    343    |    12     |     6     |     1     |
-------------------------------------------------------------
| disagree  |     8     |    71     |     0     |     0     |
-------------------------------------------------------------
|  discuss  |     3     |     1     |    889    |     5     |
-------------------------------------------------------------
| unrelated |     7     |     1     |    23     |   3628    |
-------------------------------------------------------------
Score: 2217.5 out of 2253.75	(98.39156960621187%)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    909    |    41     

In [159]:
model = trainModel(builtCompositeModel(drop_out_rate=0.5),[train_headline, train_first_body, train_last_body],[vali_headline, vali_first_body, vali_last_body])

W0723 18:07:02.809592 140219664385920 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
headline_input (InputLayer)     [(None, 15)]         0                                            
__________________________________________________________________________________________________
first_body_input (InputLayer)   [(None, 50)]         0                                            
__________________________________________________________________________________________________
headline_word_embedding_layer ( (None, 15, 300)      8825700     headline_input[0][0]             
__________________________________________________________________________________________________
last_body_input (InputLayer)    [(None, 50)]         0                                            
___________________________________________________________________________________________

In [160]:
pred = model.predict([vali_headline, vali_first_body,vali_last_body])
y_classes = pred.argmax(axis=-1)
actual = y_vali
evaluate(y_classes, actual)



pred = model.predict([competition_headline, competition_first_body, competition_last_body])
y_classes = pred.argmax(axis=-1)
actual = compe['Stance']
evaluate(y_classes, actual)

Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    342    |     8     |     8     |     4     |
-------------------------------------------------------------
| disagree  |     9     |    69     |     0     |     1     |
-------------------------------------------------------------
|  discuss  |     7     |     1     |    886    |     4     |
-------------------------------------------------------------
| unrelated |     9     |     2     |    23     |   3625    |
-------------------------------------------------------------
Score: 2211.5 out of 2253.75	(98.12534664448143%)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    847    |    35     